# Detection de frelons à l'aide de l'algorithme YoloV5

In [1]:
import torch
from IPython.display import Image, clear_output, display  # librairie d'affichage des images
from IPython.core.magic import register_line_cell_magic
%cd yolov5
from utils.downloads import gdrive_download
import yaml
import glob

/home/eisti/Desktop/ING3/usecase1/UseCase_frelon_asiatique/yolov5


# Mise en place du dataset

Après avoir collecté des images dans 3 classes différentes (Frelons asiatiques, Frelons européens et autres insectes dont abeilles, guêpes et mouches), nous les avons labellisé en utilisant RoboFlow. On obtient alors une clé API qui nous permet de récupéré des données pré-formatées que l'on pourra utiliser simplement pour entrainer notre algorithme. 

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="l7xZVOnY9MzH8MTMppTB")
project = rf.workspace().project("usecasehornet")
dataset = project.version(2).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to useCaseHornet-2 in yolov5pytorch:: 100%|██████████| 474/474 [00:00<00:00, 3478.92it/s]


In [3]:
# informations sur le dataset utilisé
%cat {dataset.location}/data.yaml

names:
- asian
- european
- other
nc: 3
train: useCaseHornet-2/train/images
val: useCaseHornet-2/valid/images


# Définition des paramètres du modèle

On définit ensuite les différents paramètres de notre modèle. 

In [4]:
# on récupère le nombre de classes de nos données
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
print("Nombre de classes :", num_classes)

Nombre de classes : 3


In [6]:
# fonction pour écrire le fichier de configuration du modèle
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [7]:
%%writetemplate models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

# Entrainement sur nos données

Une fonction permettant de réentrainer le modèle sur nos données est disponible dans le repository utilisé, dans le fichier train.py. 

On utilise les paramètres suivants : 
- **img:** taille des images d'entrée
- **batch:** taille des batchs de données
- **epochs:** nombre d'epochs
- **data:** chemin vers le fichier de données
- **cfg:** fichier de configuration du modèle
- **weights:** fichiers contenant les poids pré-entrainés
- **name:** nom du fichier de résultats
- **nosave:** pour ne sauvegarder qu'à la fin de l'entrainement
- **cache:** images gardées en cache (améliore la vitesse de l'entrainement)

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
#%%time
import train
!python train.py --img 416 --batch 12 --epochs 100 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache
#datasetloc = str({dataset.location})
#args = "--img 416 --batch 10 --epochs 2 --data "+datasetloc+"/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache"
#param = train.parse_opt(args)
#train.main(param)

train: weights=, cfg=./models/custom_yolov5s.yaml, data=/home/eisti/Desktop/ING3/usecase1/UseCase_frelon_asiatique/yolov5/useCaseHornet-2/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=100, batch_size=12, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 66a292d torch 1.10.2+cu102 CPU

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, 

      0/99        0G    0.1129   0.02059   0.03811        30       416:  50%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
      0/99        0G    0.1126   0.02075   0.03815        32       416:  56%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
      0/99        0G    0.1116   0.02117   0.03814        36       416:  62%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_t

      1/99        0G   0.09786   0.02301   0.03724        25       416:  94%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
      1/99        0G   0.09745   0.02295   0.03724        21       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         25         27     0.0017      0.411    0.00603   0.000919

     Epoch   gpu_mem       box       obj       cls    labels  img_size
  0%|          | 0/16 [00:00<?, ?it/s]                                          /home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is no

      3/99        0G   0.08626   0.02538   0.03683        34       416:  31%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
      3/99        0G   0.08689   0.02512   0.03651        30       416:  38%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
      3/99        0G   0.08607   0.02515   0.03654        31       416:  44%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_t

      4/99        0G   0.08252   0.02493   0.03654        30       416:  75%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
      4/99        0G   0.08237   0.02489   0.03644        29       416:  81%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
      4/99        0G   0.08246   0.02519   0.03645        36       416:  88%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_t

...

     21/99        0G   0.08156     0.024    0.0338        33       416:  62%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     21/99        0G   0.08094    0.0243   0.03386        33       416:  69%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     21/99        0G   0.08061   0.02456   0.03382        32       416:  75%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_t

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         25         27     0.0879     0.0944     0.0346    0.00787

     Epoch   gpu_mem       box       obj       cls    labels  img_size
  0%|          | 0/16 [00:00<?, ?it/s]                                          /home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     23/99        0G   0.07256   0.02238   0.03299        28       416:   6%|▋  /home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     23/99        0G   0.07539   0.02068   0.03614      

     24/99        0G   0.07108   0.02373   0.02829        29       416:  38%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     24/99        0G   0.07213   0.02302   0.02935        25       416:  44%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     24/99        0G   0.06991   0.02244   0.02848        26       416:  50%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_t

     25/99        0G   0.07471   0.02517   0.03267        37       416:  81%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     25/99        0G   0.07463   0.02507   0.03268        28       416:  88%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     25/99        0G   0.07521    0.0248   0.03257        27       416:  94%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_t

     27/99        0G   0.06632   0.02664   0.02929        36       416:  19%|█▉ /home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     27/99        0G   0.06628   0.02646   0.02997        28       416:  25%|██▌/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
     27/99        0G   0.06773   0.02794    0.0307        39       416:  31%|███/home/eisti/Desktop/ING3/usecase1/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_t

# Evaluation de la performance de l'algorithme

On peut visualiser les résultats (métriques etc.) de l'algorithme dans un Tensorboard. 


In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
#%load_ext tensorboard
#%tensorboard --logdir runs

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason... 
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='runs/train/yolov5s_results/results.png', width=1000)  # view results.png

# Visualisation des résultats

In [ ]:
# résultats sur le premier batch de données de validation
Image(filename='runs/train/yolov5s_results/val_batch0_labels.jpg', width=900)

In [ ]:
# puis sur le second batch
Image(filename='runs/train/yolov5s_results/val_batch1_labels.jpg', width=900)

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
# use the best weights!
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source useCaseHornet-2/test/images

In [ ]:
for imageName in glob.glob('runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")